In [1]:
import pandas as pd

## Pobranie i oczyszczenie danych pull requestów

In [2]:
pull_requests = pd.read_pickle('./../../data/00_db/pull_requests.pkl')

### Informacje o zbiorze

In [3]:
pull_requests.shape

(78955, 9)

In [4]:
pull_requests.dtypes

id                  int64
head_repo_id      float64
base_repo_id        int64
head_commit_id    float64
base_commit_id      int64
user_id             int64
pullreq_id          int64
intra_branch        int64
merged              int64
dtype: object

In [5]:
pull_requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78955 entries, 0 to 78954
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              78955 non-null  int64  
 1   head_repo_id    64864 non-null  float64
 2   base_repo_id    78955 non-null  int64  
 3   head_commit_id  64744 non-null  float64
 4   base_commit_id  78955 non-null  int64  
 5   user_id         78955 non-null  int64  
 6   pullreq_id      78955 non-null  int64  
 7   intra_branch    78955 non-null  int64  
 8   merged          78955 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 5.4 MB


In [6]:
pull_requests.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78955 entries, 0 to 78954
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              78955 non-null  int64  
 1   head_repo_id    64864 non-null  float64
 2   base_repo_id    78955 non-null  int64  
 3   head_commit_id  64744 non-null  float64
 4   base_commit_id  78955 non-null  int64  
 5   user_id         78955 non-null  int64  
 6   pullreq_id      78955 non-null  int64  
 7   intra_branch    78955 non-null  int64  
 8   merged          78955 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 5.4 MB


In [7]:
pull_requests.head()

,id,head_repo_id,base_repo_id,head_commit_id,base_commit_id,user_id,pullreq_id,intra_branch,merged
0,1,38.0,4,46.0,45,22,2,0,1
1,2,14.0,4,194.0,60,24,1,0,0
2,3,113.0,3,195.0,95,124,12,0,0
3,4,102.0,3,99.0,98,58,10,0,1
4,5,102.0,3,97.0,99,58,8,0,1


### Usuwanie zbędnych danych 

Nie będę korzystał z danych `head_repo` oraz `head_commit` - interesuje mnie tylko, z którego projektu był robiony fork, a to jest równoważne z projektem podanym w `base_repo_id`.

Nie jest natomiast dla mnie do końca jasne co kryje się pod `pullreq_id`, jednak stosowanie tego pola przy łączeniu z komentarzami daje bardzo dużo braków, a `id` nie daje żadnych - na tej podstawie wnioskuję, żeby korzystać z głównego `id` w przypadku tej tabeli, a drugą usuwam, aby nie zaciemniać obrazu danych.

In [8]:
pull_requests.isnull().sum()

id                    0
head_repo_id      14091
base_repo_id          0
head_commit_id    14211
base_commit_id        0
user_id               0
pullreq_id            0
intra_branch          0
merged                0
dtype: int64

In [9]:
pull_requests.drop(['head_repo_id', 'head_commit_id', 'base_commit_id', 'user_id', 'intra_branch', 'pullreq_id'], axis=1, inplace=True)

### Zmiana typów

In [10]:
pull_requests.dtypes

id              int64
base_repo_id    int64
merged          int64
dtype: object

Żaden typ nie wymaga zmiany

### Filtrowanie

Filtruję po polu `base_repo_id` w efekcie otrzymuję więcej wierszy spełniających warunki, niż jakbym filtrował po `head_repo_id`.

In [11]:
projects = pd.read_pickle('../../data/01_data_from_db/projects.pkl')
projects.head()

,project_id,name,language,created_at
0,1,akka,Scala,2009-02-16 12:51:54
1,2,devtools,R,2010-05-03 04:08:49
2,3,ProjectTemplate,R,2010-08-24 17:22:36
3,4,stat-cookbook,R,2012-04-23 20:24:37
4,5,hiphop-php,C++,2010-01-02 01:17:06


In [12]:
pull_requests = pull_requests[pull_requests.base_repo_id.isin(projects.project_id)]

Ponowne sprawdzenie informacji o zbiorze

In [13]:
pull_requests.shape

(78955, 3)

In [14]:
pull_requests.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78955 entries, 0 to 78954
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   id            78955 non-null  int64
 1   base_repo_id  78955 non-null  int64
 2   merged        78955 non-null  int64
dtypes: int64(3)
memory usage: 2.4 MB


Zmiana nazw kolumn

In [15]:
pull_requests.rename(columns={'id': 'pull_request_id', 'base_repo_id': 'project_id'}, inplace=True)
pull_requests.head()

,pull_request_id,project_id,merged
0,1,4,1
1,2,4,0
2,3,3,0
3,4,3,1
4,5,3,1


### Eksport do `.pkl`

In [16]:
pull_requests.to_pickle('../../data/01_data_from_db/pull_requests.pkl')